# Objetivo

O presente notebook faz parte da grade curricular da tuma 02/2022 do curso de Data Science & Machine Learning da Tera e tem como objetivo aplicar os conhecimentos adquiridos ao longo do curso.



# Problema de negócio (fictício)

O grupo fictício TERASHOP possui um e-commerce de um supermercado. Os gestores identificaram que as vendas do e-commerce não estavam performando conforme o esperado e demandaram à equipe de pesquisa que identificassem os principais ofensores.

Em seus estudos, foi identificado que os pedidos possuíam um volume de produtos abaixo do esperado em comparação com as previsões de crescimento do negócio, o que impacta diretamente o valor total de cada carrinho de compras bem como a receita total do e-commerce.

Iniciou-se então uma investigação para tentar mapear as necessidades do negócio a partir da constatação de que o volume de produtos nos pedidos estava abaixo do esperado.

Uma das oportunidades de melhoria identificadas neste processo de investigação foi a implementação de um mecanismo que pudesse fazer com que os consumidores conhecessem melhor a variedade de produtos oferecidos pelo e-commerce ao mesmo tempo em que proporcionasse uma experiência positiva ao navegar pelo site e tivesse a prerrogativa de influenciar o perfil de compras do usuário.

## Solução

Aplicação de Regras de Associação para o banco de dados [InstaCart](https://www.kaggle.com/competitions/instacart-market-basket-analysis/data?select=order_products__train.csv.zip).

As regras de Associação têm como objetivo encontrar elementos que implicam na presença de outros elementos em uma mesma transação, ou seja, encontrar relacionamentos ou padrões frequentes entre conjuntos de dados. Como exemplo, {𝑙𝑒𝑖𝑡𝑒,𝑝ã𝑜}→{𝑜𝑣𝑜𝑠} é uma associação que diz que quando se encontra os itens 𝑙𝑒𝑖𝑡𝑒 e 𝑝ã𝑜 em uma ocorrência, é esperado que o 𝑜𝑣𝑜𝑠 também apareçam na transação.

## Descrição da Base de Dados

O conjunto de dados, distribuídos em 6 arquivos .csv, contém uma amostra de mais de 3 milhões de pedidos de supermercado, com histórico de aproximadamente 200.000 usuários do Instacart, empresa americana que opera um serviço de entrega e coleta de alimentos nos Estados Unidos e no Canadá. A empresa oferece seus serviços por meio de um site e também por aplicativo móvel.

A fonte de dados conta com os seguintes arquivos:

products.csv: contém os nomes dos produtos com seu product_id correspondente. Além disso, o corredor e o departamento estão inclusos no conjunto de dados.

orders.csv: fornece uma lista de todos os ids de pedidos no conjunto de dados, ou seja, uma linha para cada pedido, com a identificação de qual arquivo ele pertence (train ou prior).

departaments.csv: traz informações sobre a identificação dos diferentes departamentos.

order_products_*.csv: fornecem informações sobre quais produtos (product_id) foram comprados em cada order_id.



# Importando Bibliotecas

In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori

# Leitura das Bases de Dados

In [2]:
# Base com as transações
df=pd.read_csv(r"C:\Users\olima\Documents\Python\association-rules\order_products__train.csv")

# Base com os nomes dos produtos
prod=pd.read_csv(r"C:\Users\olima\Documents\Python\association-rules\products.csv")

# Base com informações dos pedidos
orders=pd.read_csv(r"C:\Users\olima\Documents\Python\association-rules\orders.csv")


# Exploração das Bases de Dados

In [3]:
df.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384617 entries, 0 to 1384616
Data columns (total 4 columns):
 #   Column             Non-Null Count    Dtype
---  ------             --------------    -----
 0   order_id           1384617 non-null  int64
 1   product_id         1384617 non-null  int64
 2   add_to_cart_order  1384617 non-null  int64
 3   reordered          1384617 non-null  int64
dtypes: int64(4)
memory usage: 42.3 MB


## Verificando missings

In [5]:
df.order_id.isna().sum()

0

In [6]:
df.product_id.isna().sum()

0

## Verificando a quantidade de Ids únicos

In [7]:
df.order_id.nunique()

131209

In [8]:
df.product_id.nunique()

39123

In [9]:
df.order_id.value_counts()

1395075    80
2813632    80
949182     77
2869702    76
341238     76
           ..
1144944     1
1144765     1
1144608     1
1144038     1
3214874     1
Name: order_id, Length: 131209, dtype: int64

In [10]:
df.product_id.value_counts()

24852    18726
13176    15480
21137    10894
21903     9784
47626     8135
         ...  
42744        1
5871         1
47237        1
9305         1
38900        1
Name: product_id, Length: 39123, dtype: int64

# Transformação das Bases de Dados

Em um primeiro momento, foram excluídos todos os pedidos que tinham apenas um produto, tendo em vista que não esses pedidos não contribuem para o modelo, já que não é possível inferir que clientes que levam um produto determinado, também levam um outro produto.

Após, foram filtrados produtos que apareceram na base dados menos de 200 vezes, para priorizar os produtos que já apresentam um bom volume de vendas. Esse valor foi determinado de forma arbitrária, de forma a possibilitar que o algoritmo fosse executado, por questões de limitação de memória.

Neste caso, vale a pena explorar adaptações que otimizam a execução do algoritmo a priori por meio de amostragem de itens, partição dos dados, hash-based, etc.


## Filtros

In [11]:
filtro_orderId=list(df.order_id.value_counts()[df.order_id.value_counts()>1].index)
filtro_produtos=list(df.product_id.value_counts()[df.product_id.value_counts()>200].index)

df=df[df['order_id'].isin(filtro_orderId)]
df=df[df['product_id'].isin(filtro_produtos)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 756896 entries, 2 to 1384616
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype
---  ------             --------------   -----
 0   order_id           756896 non-null  int64
 1   product_id         756896 non-null  int64
 2   add_to_cart_order  756896 non-null  int64
 3   reordered          756896 non-null  int64
dtypes: int64(4)
memory usage: 28.9 MB


In [12]:
df.order_id.nunique() # Quantidade de pedidos restantes

117745

In [13]:
df.product_id.nunique() # quantidade de produtos restantes


1148

## Join com a base de nome dos produtos

In [14]:
prod.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [15]:
data=df.merge(prod,left_on="product_id",right_on="product_id")
data.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,1,10246,3,0,Organic Celery Hearts,83,4
1,2869,10246,4,1,Organic Celery Hearts,83,4
2,3378,10246,19,0,Organic Celery Hearts,83,4
3,14119,10246,6,0,Organic Celery Hearts,83,4
4,17152,10246,22,1,Organic Celery Hearts,83,4


In [16]:
# Join com as informações sobre o pedido
data=data.merge(orders,left_on="order_id",right_on="order_id")

In [17]:
data.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,10246,3,0,Organic Celery Hearts,83,4,112108,train,4,4,10,9.0
1,1,49683,4,0,Cucumber Kirby,83,4,112108,train,4,4,10,9.0
2,1,13176,6,0,Bag of Organic Bananas,24,4,112108,train,4,4,10,9.0
3,1,47209,7,0,Organic Hass Avocado,24,4,112108,train,4,4,10,9.0
4,1,22035,8,1,Organic Whole String Cheese,21,16,112108,train,4,4,10,9.0


In [18]:
# Mantendo apenas order_id e product_name
data=data.iloc[:,[0,4]][data.order_dow==0]
data.head()

,order_id,product_name
22,14119,Organic Celery Hearts
23,14119,Organic Unsweetened Almond Milk
24,14119,Cherubs Heavenly Salad Tomatoes
25,14119,Organic Baby Carrots
26,14119,Organic Baby Spinach


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185422 entries, 22 to 756892
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   order_id      185422 non-null  int64 
 1   product_name  185422 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.2+ MB


In [21]:
data.product_name.value_counts().head(10)

Banana                    4692
Bag of Organic Bananas    3549
Organic Strawberries      2798
Organic Baby Spinach      2636
Large Lemon               2218
Organic Avocado           2027
Organic Hass Avocado      1892
Limes                     1607
Strawberries              1555
Organic Raspberries       1376
Name: product_name, dtype: int64

Dentre os produtos mais pedidos, é possível verificar que existem variações de um mesmo produto, por exemplo, banana, Bag Of banana, organic banana. Esses termos foram filtrados. Como sugestão, seria interessante utilizar alguma técnica de processamento de linguagem natural para verificar a similaridade entre as palavras, mas para fins deste exercício, manteve-se o filtro apenas pelo observado.

Ainda, aqui vale observar que informação da área de negócio é essencial para determinar se realmente faz sentido juntar os produtos em um só, ou se seria interessante manter as regras separadas.

In [23]:
substituicao={'Organic ':'','Box of ':'','Bag of':'','Bananas':'Banana','Hass':''}
data['product_name']=data['product_name'].replace(substituicao,regex=True).str.strip()
data.product_name.value_counts().head(10)

Banana          8929
Avocado         4399
Strawberries    4353
Baby Spinach    2947
Large Lemon     2218
Raspberries     2155
Blueberries     1739
Garlic          1616
Limes           1607
Whole Milk      1496
Name: product_name, dtype: int64

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185422 entries, 22 to 756892
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   order_id      185422 non-null  int64 
 1   product_name  185422 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.2+ MB


## One hot encoding

Para executar o algoritmo Apriori é necessário que o banco de dados esteja organizado de tal forma que as linhas representem os pedidos e as colunas são a lista de cada produto nos pedidos. Caso um produto tenha sido solicitado no pedido, deve ser tratado com 1, caso contrário, será 0 (one hot enconding).

In [25]:
basket= data[['order_id','product_name']].value_counts().unstack().reset_index().fillna(0).set_index('order_id')
basket.head()

product_name,& Raw Strawberry Serenity Kombucha,0% Greek Strained Yogurt,1% Low Fat Milk,1% Lowfat Milk,100 Calorie Per Bag Popcorn,100% Apple Juice,100% Grated Parmesan Cheese,100% Lactose Free Fat Free Milk,100% Natural Spring Water,100% Pure Apple Juice,...,"Yogurt, Lowfat, Strawberry","Yogurt, Strained Low-Fat, Coconut",Yokids Lemonade/Blueberry Variety Pack Yogurt Squeezers Tubes,Yukon Gold Potatoes 5lb Bag,ZBar Chocolate Brownie Energy Snack,Zero Calorie Cola,Zucchini,Zucchini Spirals,Zucchini Squash,smartwater® Electrolyte Enhanced Water
order_id,,,,,,,,,,,,,,,,,,,,,
170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
218,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
226,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
def encode_units(x):
    if x <= 0:
        return False
    if x >= 1:
        return True

basket_sets = basket.applymap(encode_units)

In [27]:
basket_sets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25517 entries, 170 to 3421063
Columns: 1054 entries, & Raw Strawberry Serenity Kombucha to smartwater® Electrolyte Enhanced Water
dtypes: bool(1054)
memory usage: 25.8 MB


# Criação das Regras

Com os dados devidamente estruturados, é possível aplicar o algoritmo para cálculo dos itens frequentes, que é a base para a criação das regras. Consiste em verificar os produtos ou conjunto de produtos que aparecem nos pedidos de acordo com um limite mínimo pré-estabelecido.

Para este estudo, limitou-se aos produtos com **suporte** de 1%, ou seja, que possuem frequência relativa de 1% no total da base de dados. Esse valor foi escolhido de forma arbitrária, mas pode ser alterado para verificar o que acontece. Neste caso, não se observou muita diferença nas regras executadas.

In [28]:
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True,low_memory=True)
frequent_itemsets.sort_values('support',ascending=False)

,support,itemsets
13,0.349218,(Banana)
6,0.172042,(Avocado)
126,0.170004,(Strawberries)
11,0.115413,(Baby Spinach)
231,0.088451,"(Banana, Strawberries)"
...,...,...
224,0.010150,"(Banana, Roasted Turkey Breast)"
263,0.010072,"(Banana, Cucumber, Avocado)"
177,0.010072,"(Baby Carrots, Strawberries)"
209,0.010033,"(Banana, Italian Parsley Bunch)"


Suporte máximo obtido de 0,34

Restringindo a produtos que apareceram 500 vezes, o suporte máximo foi de 0,36. 


In [29]:
# Criação das Regras
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Banana),(100% Whole Wheat Bread),0.349218,0.023083,0.010699,0.030636,1.327243,0.002638,1.007792
1,(100% Whole Wheat Bread),(Banana),0.023083,0.349218,0.010699,0.463497,1.327243,0.002638,1.213008
2,(Banana),(2% Reduced Fat Milk),0.349218,0.025943,0.010228,0.029290,1.128979,0.001169,1.003447
3,(2% Reduced Fat Milk),(Banana),0.025943,0.349218,0.010228,0.394260,1.128979,0.001169,1.074358
4,(Banana),(Apple Honeycrisp Organic),0.349218,0.021397,0.012070,0.034564,1.615330,0.004598,1.013638
...,...,...,...,...,...,...,...,...,...
295,"(Banana, Strawberries)",(Raspberries),0.088451,0.084218,0.018145,0.205140,2.435806,0.010696,1.152129
296,"(Raspberries, Strawberries)",(Banana),0.030960,0.349218,0.018145,0.586076,1.678252,0.007333,1.572225
297,(Banana),"(Raspberries, Strawberries)",0.349218,0.030960,0.018145,0.051958,1.678252,0.007333,1.022149
298,(Raspberries),"(Banana, Strawberries)",0.084218,0.088451,0.018145,0.215449,2.435806,0.010696,1.161874


In [30]:
rules.sort_values(by='lift',ascending=False).to_csv('teste.csv')

In [31]:
rules.lift.mean()

1.6940066333742185

## Interpretações

- As regras de acordo com os critérios estabelecidos representam o perfil de consumo de produtos horti-fruti granjeiros. Todas as regras envolvem esses produtos.

- Exemplifica-se a interpretação da última regra: quem compra morangos (antecedente) terá como sugestão levar também banana e raspberries (consequentes). 
    - O suporte do antecedente significa que morangos foram comprados em 17% dos pedidos considerados. O suporte do consequente significa que o item frequente composto por banana e raspberries apareceu em 4% dos pedidos. Já o suporte de 1,8% é o percentual de vezes que a regra aconteceu, ou seja, que foram comprados os 3 produtos. 
    - confiança refere-se ao percentual de vezes em que bananas e raspberries foram compradas, dado que morangos foram compradas, ou seja, 10% dos pedidos que contêm morangos, também contém bananas e raspberries.
    - O lift foi tomado como a medida mais importante, ela representa a chance de um pedido que contém morango, também conter banana e raspberries. No caso, o cliente que leva banana tem 2,5 vezes mais chances de também levar bananas e raspberries.
    - Conviction representa o percentual de falha da regra.

# Extra

Nesta etapa, testou-se se a criação de regras por dia da semana ou por departamento traduz alguma diferença nas regras, o que pode se beneficiar de um conhecimento técnico na área de negócio. Neste estudo de caso, serviu apenas como um exercício.

In [37]:
filtro_orderId=list(df.order_id.value_counts()[df.order_id.value_counts()>1].index)
filtro_produtos=list(df.product_id.value_counts()[df.product_id.value_counts()>200].index)
# data=df.merge(prod,left_on="product_id",right_on="product_id")
# data=data.merge(orders,left_on="order_id",right_on="order_id")


for i in range(0,7):
    data=df.merge(prod,left_on="product_id",right_on="product_id")
    data=data.merge(orders,left_on="order_id",right_on="order_id")
    data=data[(data['order_id'].isin(filtro_orderId)) & (data.order_dow==i)]
    data=data[data['product_id'].isin(filtro_produtos)]
    data=data.loc[:,['order_id','product_name']]#[data.order_dow==i]
    substituicao={'Organic ':'','Box of ':'','Bag of':'','Bananas':'Banana','Hass':''}
    data['product_name']=data['product_name'].replace(substituicao,regex=True).str.strip()
    basket= data[['order_id','product_name']].value_counts().unstack().reset_index().fillna(0).set_index('order_id')

    basket_sets = basket.applymap(encode_units)

    frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True,low_memory=True)
    frequent_itemsets.sort_values('support',ascending=False)
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
    rules.sort_values(by='lift',ascending=False).to_csv(f'regras_{i}.csv')
    print("DOW: ",i)
    print(rules.lift.mean())


DOW:  0
1.6103061416860593
DOW:  1
1.633945897043864
DOW:  2
1.7385429942513986
DOW:  3
1.774693897027502
DOW:  4
1.8024558639919754
DOW:  5
1.8157363194041574
DOW:  6
1.7729557668339988


In [38]:
filtro_orderId=list(df.order_id.value_counts()[df.order_id.value_counts()>1].index)
filtro_produtos=list(df.product_id.value_counts()[df.product_id.value_counts()>200].index)
# data=df.merge(prod,left_on="product_id",right_on="product_id")
# data=data.merge(orders,left_on="order_id",right_on="order_id")


for i in prod.department_id.unique():
    data=df.merge(prod,left_on="product_id",right_on="product_id")
    data=data.merge(orders,left_on="order_id",right_on="order_id")
    data=data[(data['order_id'].isin(filtro_orderId)) & (data.department_id==i)]
    if(data.shape[0]>0):
        data=data[data['product_id'].isin(filtro_produtos)]
        data=data.loc[:,['order_id','product_name']]#[data.order_dow==i]
        substituicao={'Organic ':'','Box of ':'','Bag of':'','Bananas':'Banana','Hass':''}
        data['product_name']=data['product_name'].replace(substituicao,regex=True).str.strip()
        basket= data[['order_id','product_name']].value_counts().unstack().reset_index().fillna(0).set_index('order_id')

        basket_sets = basket.applymap(encode_units)

        frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True,low_memory=True)
        frequent_itemsets.sort_values('support',ascending=False)
        rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
        rules.sort_values(by='lift',ascending=False).to_csv(f'regras_dep_{i}.csv')
        print("Department: ",i)
        print(rules.lift.mean())
    


Department:  19
nan
Department:  13
nan
Department:  7
2.854541738999599
Department:  1
3.9889776665383136
Department:  11
nan
Department:  16
nan
Department:  17
nan
Department:  18
1.2328925766907228
Department:  12
nan
Department:  9
nan
Department:  14
nan
Department:  15
nan
Department:  4
1.5238056166753247
Department:  21
nan
Department:  6
nan
Department:  20
nan
Department:  5
nan
Department:  3
nan
Department:  10
nan


# Conclusão

O algoritmo de regra de associação Apriori é um dos mais simples, de fácil implementação, e ainda assim fornece conhecimentos valiosos sobre os dados e comportamentos de compras em um e-commerce. Entretanto, para futuro aprimoramento do modelo, pode-se pensar em um modelo para sistema de recomendação baseado em conteúdo por cliente, colaborativo ou até mesmo o híbrido.


# Referência

https://medium.com/@marinhoruan71/regras-de-associa%C3%A7%C3%A3o-em-um-e-commerce-de-mercado-6def22a63b27
